Following the zhinst-toolkit documentation about how the AWG sequencer works.

See https://docs.zhinst.com/zhinst-toolkit/en/latest/examples/awg.html for the reference document

Next complete
https://docs.zhinst.com/shfqc_user_manual/tutorials/tutorial_command_table.html

In [2]:
from zhinst.toolkit import Session, CommandTable, Waveforms
import random
import numpy as np
from IPython.display import display, clear_output
import time

import matplotlib.pyplot as plt

In [9]:
dataserver_ip = "10.1.1.1"
dataserver_port = 8004
device_id = "DEV12158"

session = Session(dataserver_ip, dataserver_port)
device = session.connect_device(device_id)
fs = 2e9

device.system.preset.load(1) 
device.system.preset.load.wait_for_state_change(0)  # block until we actually load the system

device

CoreError: Could not connect to a Zurich Instruments data server on host '10.1.1.1' and port 8004. Make sure that the server is running, host / port names are correct, API / DataServer versions match. Execution of connect took longer than 20 seconds.

In [13]:
awg_node = device.sgchannels[0].awg  # breakout the AWG for ease of use

In [8]:
# copied code. should be able to run without changes?
# must subscribe to the result node before readout
def get_results(result_node, timeout):
    wave_data_captured = {}
    wave_data_captured[result_node] = False
    start_time = time.time()
    captured_data = {}
    while not all(wave_data_captured.values()):
        if start_time + timeout < time.time():
            raise TimeoutError('Timeout before all samples collected.')
        test = session.poll()
        for node, value in test.items():
            node = session.raw_path_to_node(node)
            for v in value:
                if node not in captured_data:
                    captured_data[node] = [v['vector']]
                else:
                    captured_data[node].append(v['vector'])
            if len(captured_data[node]) >= 1: # readout 1 point
                wave_data_captured[node] = True
                total_num_data = sum([len(element) for element in captured_data[node]])
    data = captured_data[result_node][0]
    return data

In [ ]:
# Create maps to channels for simplicity

chan = {
    "measure": device.qachannels[0],  # measure and acquire lines
    "st": device.sgchannels[0],  # drive SET (ST) line
    "p1": device.sgchannels[1],  # drive
    "p2": device.sgchannels[2],  # drive
    "j1": device.sgchannels[3],  # drive  
}

drive_chans = ["st", "p1", "p2", "j1"]  # match keys above

In [12]:
session

DataServerSession(localhost:8004)

## Sequencer

Must first compile the code, then upload it to the device

Example code
```c
// Waveform paramaters
const WFM_LEN = 1008;
const GAUSS_CENTER = WFM_LEN/2;
const SIGMA = WFM_LEN/8;

// Define waveforms
wave w0_1 = gauss(WFM_LEN, 1.0, GAUSS_CENTER, SIGMA) + marker(128, 1);
wave w0_2 = drag(WFM_LEN, 1.0, GAUSS_CENTER, SIGMA);
wave w1_1 = gauss(WFM_LEN, 0.5, GAUSS_CENTER, SIGMA) + marker(128, 1);
wave w1_2 = drag(WFM_LEN, 0.5, GAUSS_CENTER, SIGMA);

// Assign waveforms to an index in the waveform memory
assignWaveIndex(1,2, w0_1, 1,2, w0_2, 0);
assignWaveIndex(1,2, w1_1, 1,2, w1_2, 1);

// Play wave 1
playWave(1,2, w0_1, 1,2, w0_2);
// Play wave 2
playWave(1,2, w1_1, 1,2, w1_2);
```

In [14]:
from zhinst.toolkit import Sequence

In [15]:
seq = Sequence()
seq.code = """\
// Hello World
repeat(5)
...
"""
seq.constants["PULSE_WIDTH"] = 10e-9 #ns
print(seq)

#awg_node.load_sequencer_program(SEQUENCER_CODE)  # compile and upload

// Constants
const PULSE_WIDTH = 1e-08;
// Hello World
repeat(5)
...



## Waveform

Specify the actual waves (samples) to play. Need to give two waves (I and Q? Although we only have one output)

Make sure that the waves are already defined in the sequence program before uploading to the device.

The `waveform.index` or `slot` is just the position in the memory of where the waveform is stored. We must match it with `ct.table[?].waveform.index` later to play the right wave

Full path is `<DEVICE_ID>/SGCHANNELS/<SG_CHAN_INDEX>/AWG/WAVEFORM/WAVES/<WAVE_INDEX>`

In [21]:
from zhinst.toolkit.waveform import Wave

In [25]:
wave01 = Wave(np.ones(1008), name="demo")

waveforms = Waveforms()
waveforms.assign_waveform(
    slot= 0, # index in assignWaveIndex
        wave1= wave01,
        wave2= wave01
)

# Before upload get sequence code
print(waveforms.get_sequence_snippet())

wave demo = placeholder(1008, false, false);
wave demo = placeholder(1008, false, false);
assignWaveIndex(demo, demo, 0);


In [27]:
# join to existing sequence
seq.waveforms = waveforms
print(seq)

// Constants
const PULSE_WIDTH = 1e-08;
// Waveforms declaration
wave demo = placeholder(1008, false, false);
wave demo = placeholder(1008, false, false);
assignWaveIndex(demo, demo, 0);
// Hello World
repeat(5)
...



In [ ]:
# Upload and download is easy
awg_node.write_to_waveform_memory(waveforms)

waveforms_device = awg_node.read_from_waveform_memory()
waveforms_device[0]

In [31]:
# verify that everything is correct.
# this is not run automatically but will test if waveforms match

waveforms = awg_node.read_from_waveform_memory()
waveforms.validate(awg_node.waveform.descriptors())  # doesn't do anything?

## Command Table

Design instructions to execute simultaniously within one clock cycle of 3.33ns

Additional instructions (commands) for how exactly to play the wave. Must be linked to one waveform index.

Full path is `<DEVICE_ID>/SGCHANNELS/<SG_CHAN_INDEX>/AWG/COMMANDTABLE/DATA`

In [39]:
from zhinst.toolkit import CommandTable

In [ ]:
# use the current schema 
ct_schema = awg_node.commandtable.load_validation_schema()
ct = CommandTable(ct_schema)

# or load it directly from the device
ct = awg_node.commandtable.load_from_device()

In [ ]:
# debugging can be done with .info()
ct.table[0].waveform.info()  # to get all 
ct.table[0].waveform.info("index")  # to just get one

# unlike the node tree we get and set directly (think as DICT/JSON structure)
ct.table[0].waveform.index = 0  # set
ct.table[0].waveform.index  # get

In [ ]:
# represent it nicely
ct.as_dict()  

## Multiple Sequencers

Sequencer code for each line needs to be created and uploaded individually. Just do the same multiple times

In [ ]:
# now get the AWG nodes for each sequncer (driver)
awg_nodes = [sgchannel.awg for sgchannel in device.sgchannels]

In [ ]:
from textwrap import dedent  # remove indents from the written code
num_cores = len(awg_nodes)
sequences = [
    dedent(f"""\
    // Core {i:d} sequence
    waitDigTrigger(1);  //Wait for a trigger to syncronize all the cores
    playWave(ramp(1024, 0, {(i+1)/num_cores:f}));
    """)
    for i in range(num_cores)
]

In [ ]:
# upload to the device.
# This is done sequentially, it is possible to do it in parallel but we'll cross that bridge if we need it

from zhinst.core.errors import CoreError
with session.set_transaction():
    for index, (awg_node, sequence) in enumerate(zip(awg_nodes, sequences)):
        try:
            _ = awg_node.load_sequencer_program(sequence)
        except CoreError as e:
            print("Compilation error on core", index, e)
            break